In [3]:
import os
#from test_helper import Test

# Change to the location of data files
# use ml-latest-small first; then on ml-latest if everything works well
dbfs_dir = 's3://geomapping/outputfolder'
devicestatus_filename = dbfs_dir + '/devicestatus.csv'
movies_filename = dbfs_dir + '/movies.csv'

In [4]:
import pyspark

In [5]:
AWS_S3_BUCKET = os.getenv("AWS_S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = os.getenv("AWS_SESSION_TOKEN")

In [6]:
sc = pyspark.SparkContext()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/03 16:20:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
import pandas as pd
devicestatus_df = pd.read_csv(
    f"s3a://geomapping/outputfolder/devicestatus.csv",
    storage_options={
        "key": AWS_ACCESS_KEY_ID,
        "secret": AWS_SECRET_ACCESS_KEY,
        "token": AWS_SESSION_TOKEN,
    }
)


In [8]:
print(devicestatus_df.columns)

Index(['Unnamed: 0', 'Longitude', 'Latitude', 'date', 'Manufacturer',
       'Model_Name', 'deviceid'],
      dtype='object')


In [9]:
alldata=devicestatus_df.drop(columns=['Unnamed: 0'])

In [10]:
alldata.head()
import numpy as np

In [11]:
alldata.replace(to_replace=[None], value=np.nan, inplace=True)
print(alldata.isnull().sum())

Longitude       0
Latitude        0
date            0
Manufacturer    0
Model_Name      0
deviceid        0
dtype: int64


In [14]:
from pyspark.sql import SQLContext
sqlcontext = SQLContext(sc)
s_df = sqlcontext.createDataFrame(alldata)

In [22]:
convergeDist = float(0.1)
iterationDist = float("inf")
distanceMeasure= "euclidean"
k=5

In [25]:
rdd=s_df.rdd

In [52]:
from point_utils import *

In [32]:
points =rdd.map(lambda line: [latlon(line[0], line[1]), line[2:]]).cache()

   

In [33]:
from fix_latlon import *

In [48]:
centroids = points.map(lambda x: x[0]).takeSample(False, k, 1)
centroids = dict(zip(range(0, k), centroids))

22/05/03 16:25:51 WARN TaskSetManager: Stage 5 contains a task of very large size (1326 KiB). The maximum recommended task size is 1000 KiB.
22/05/03 16:25:51 WARN TaskSetManager: Stage 6 contains a task of very large size (1326 KiB). The maximum recommended task size is 1000 KiB.


In [49]:
convergeDist = 1
sumDist = 2
measure ='euclidean'
kPoints=points.map(lambda point: point[0]).takeSample(False, k,1)

22/05/03 16:25:54 WARN TaskSetManager: Stage 7 contains a task of very large size (1326 KiB). The maximum recommended task size is 1000 KiB.
22/05/03 16:25:55 WARN TaskSetManager: Stage 8 contains a task of very large size (1326 KiB). The maximum recommended task size is 1000 KiB.


In [117]:
while tempDist > convergeDist:
    closest = points.map(lambda p: (closestPoint(p, kPoints), (p, 1)))
    pointStats = closest.reduceByKey(lambda p1_c1, p2_c2: (p1_c1[0] + p2_c2[0], p1_c1[1] + p2_c2[1]))
    newPoints = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()

    for (iK, p) in newPoints:
        kPoints[iK] = p
            
    if distanceMeasure == "greatcircle":
        distances = newMeans.map(lambda idx, mean: greatCircleDistance(centroids[idx], newPoints))
    else:
        distances = newMeans.map(lambda idx, mean: euclideanDistance(centroids[idx], newPoints))

        # Use the sum of the distances between old and new centroids as the convergeDist measure
    iterationDist = distances.sum()
    centroids = newPoints.collectAsMap()
print("Final centers: " + str(kPoints))

Final centers: [<fix_latlon.latlon object at 0x7fa05f95fc90>, <fix_latlon.latlon object at 0x7fa05f95f1d0>, <fix_latlon.latlon object at 0x7fa05f95b910>, <fix_latlon.latlon object at 0x7fa05f95f890>, <fix_latlon.latlon object at 0x7fa05f95ff90>]


In [108]:
result_b = []
for idx, centroid in centroids.items():
        print (str(idx) + "," + str(centroid.lat) + "," + str(centroid.lon))
        result_b.append(str(idx) + "," + str(centroid.lat) + "," + str(centroid.lon)) 


0,-118.081348964,34.6430342263
1,-111.487393105,32.0851930123
2,-119.268818405,36.8064952669
3,-117.203004571,34.6103338865
4,-122.416687051,42.8749132937


In [109]:
result_b
df = pd.DataFrame(result_b)
df[['index','Latitude','Longitude']]=df[0].str.split(',', expand=True)

In [110]:
df=df.drop(columns=[0])

In [150]:
df= df.astype(float)

In [151]:
df.dtypes

index        float64
Latitude     float64
Longitude    float64
dtype: object